In [6]:
import asyncio
import os

import dspy
from dotenv import load_dotenv, find_dotenv
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.utilities.wikidata import WikidataAPIWrapper

from agent.utils.tools import BingSearchTool
from agent.utils.tools import WikidataTool, GoogleSearchTool, WikipediaTool, GoogleKnowledgeGraphTool, calculator, \
	python_interpreter

_ = load_dotenv(find_dotenv())

In [20]:
class Plan(dspy.Signature):
	"""Given a question or problem, generate a solving plan, which is a list of steps.\
	It doesn't need to actually solve the problem, just generate a plan.\
	Based on the overall question and plan, identify all the error prone points in the problem solving process of each step.\
	These error-prone points are the key to guiding the selection of tools and the input of the selected tools.\
	Remember that there may be multiple error-prone areas in each step, and you should list them all."""
	question: str = dspy.InputField()
	plan_and_guidance: list[dict] = dspy.OutputField(desc="Each item contains a key 'step' and a key 'guidance'.")


class Tool(dspy.Signature):
	"""Regarding current step and the guidance, choose the appropriate tools to help you complete current step.\
	 The guidance should be guiding principles to guide subsequent behavior. \
	 Before providing the selected tools and corresponding inputs, \
	 you should fully understand the functions of each tool and their specific requirements for the input content. \
	 The tool name must accurately match the existing tool (including characters such as uppercase and lowercase, spaces, etc.). \
	 Remember that there may be multiple tools that meet the requirements, and there are also some tools that are not suitable for the current step. \
	 Avoid selecting tools that are obviously unsuitable.

	You have the following tool set:
	[wiki_data] - A wrapper around Wikidata. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be the exact name of the item you want information about or a Wikidata QID.
	[google_search] - A search engine. useful for when you need to answer questions about current events. Input should be a search query.
	[bing_search] - A search engine. useful for when you need to answer questions about current events. Input should be a search query.
	[calculator] - Useful when you need to calculate the value of a mathematical expression, including basic arithmetic operations. Use this tool for math operations. Input should strictly follow the numuxpr syntax.
	[python_interpreter] - Useful when you need to execute a code and get the value of the variables <answer>. Use this tool for code execution.This tool will return the code execution result and final value of the variable <answer>. Input should be pure python code string.
	[wikipedia] - A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.
	[google_knowledge_graph] - This tool searches for entities in the Google Knowledge Graph. It provides information about people, places, things, and concepts. Useful when you need to get information about a specific entity. Input should be an entity name.
	"""
	past_steps: list[dict] = dspy.InputField(desc="The past steps and guidance, ordered by the order of the steps.")
	current_step_and_guidance: dict = dspy.InputField(desc="The current step and guidance.")
	tool: dict = dspy.OutputField(
		desc="List all the appropriate tools to help you complete current step."
		     "Each item is a dictionary, the key is the tool name, the value is the input content of the tool.")


class Result(dspy.Signature):
	"""Based on current step, guidance and tool result, provide your answer to the current step. Remember that all the tool result are reliable. If the tool result contains the information you need, you should list the information as evidence for the current step."""

	step: str = dspy.InputField()
	guidance: str = dspy.InputField()
	tool_result: list[dict] = dspy.InputField()
	evidence: str = dspy.OutputField(
		desc="The evidence contains the information in the tool result which is needed for the current step answer.")
	answer: str = dspy.OutputField(desc="The answer to the current step.")


class FinalAnswer(dspy.Signature):
	"""Based on the question、steps、evidence and results, integrate all information that can help solve the problem and then provide your final answer. \
	Your response should contains two part, the first part is the fusion of the Revising Process and the second part is the final answer."""

	question: str = dspy.InputField()
	steps: list[dict] = dspy.InputField(desc="Each item contains a key 'step', a key 'evidence' and a key 'result'.")
	fusion: str = dspy.OutputField(desc="The fusion of the Revising Process.")
	final_answer: str = dspy.OutputField(desc="The final answer to the question.")

In [7]:
wikidata_tool = WikidataTool(api_wrapper=WikidataAPIWrapper())
google_search_tool = GoogleSearchTool()
bing_search_tool = BingSearchTool()
wikipedia_tool = WikipediaTool(api_wrapper=WikipediaAPIWrapper())
google_knowledge_graph_tool = GoogleKnowledgeGraphTool(api_key=os.environ.get("GOOGLE_API_KEY"))
tool_set = {"wiki_data": wikidata_tool, "google_search": google_search_tool, "bing_search": bing_search_tool,
            "calculator": calculator, "python_interpreter": python_interpreter, "wikipedia": wikipedia_tool,
            "google_knowledge_graph": google_knowledge_graph_tool}
tool_description = "\n".join([f"[{tool_name}] - {tool.description}" for tool_name, tool in tool_set.items()])


async def tool_ainvoke(tool_dictionary: dict):
	async def run_tool(tool_name, tool_input):
		try:
			result = await tool_set[tool_name].arun(tool_input)
			return {"tool_name": tool_name, "tool_input": tool_input, "tool_result": result}
		except Exception as e:
			return {"tool_name": tool_name, "tool_input": tool_input, "tool_result": str(e)}

	tasks = [run_tool(tool_name, tool_input) for tool_name, tool_input in tool_dictionary.items()]

	tool_result = await asyncio.gather(*tasks)
	return tool_result


print(tool_description)

[wiki_data] - A wrapper around Wikidata. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be the exact name of the item you want information about or a Wikidata QID.
[google_search] - A search engine. useful for when you need to answer questions about current events. Input should be a search query. 
[bing_search] - A search engine. useful for when you need to answer questions about current events. Input should be a search query. 
[calculator] - Useful when you need to calculate the value of a mathematical expression, including basic arithmetic operations. Use this tool for math operations. Input should strictly follow the numuxpr syntax.
[python_interpreter] - Useful when you need to execute a code and get the value of the variables <answer>. Use this tool for code execution.This tool will return the code execution result and final value of the variable <answer>. Input should be pure python c

In [24]:
dspy.configure(lm=dspy.LM(model='openai/gpt-4o-mini', temperature=0, api_base="https://api.chsdw.top/v1"))
plan = dspy.Predict(Plan)
tool = dspy.ChainOfThought(Tool)
cot = dspy.Predict(Result)

question = "Who was known by his stage name Aladin and helped organizations improve their performance as a consultant?"
plan_result = plan(question=question)
print(plan_result)
tool_selection = tool(past_steps=[],
                      current_step_and_guidance={"current_step": plan_result.plan_and_guidance[0]["step"],
                                                 "guidance": plan_result.plan_and_guidance[0]["guidance"]})
print(tool_selection)
tool_result = await tool_ainvoke(tool_selection.tool)
print(tool_result)
cot_result = cot(step=plan_result.plan_and_guidance[0]["step"], guidance=plan_result.plan_and_guidance[0]["guidance"],
                 tool_result=tool_result)
print(cot_result)

Prediction(
    plan_and_guidance=[{'step': 'Identify the person known as Aladin', 'guidance': 'Research the stage name Aladin to find out who it refers to in the context of organizational consulting.'}, {'step': "Find details on the individual's consulting work", 'guidance': 'Look for information regarding how the person has helped organizations improve their performance, including projects, methodologies, and expertise.'}, {'step': 'Explore contributions and impact', 'guidance': "Investigate specific contributions or improvements attributed to this individual's consulting work and gather case studies or testimonials."}, {'step': 'Validate the information', 'guidance': 'Cross-check the gathered information from multiple sources to ensure accuracy and credibility regarding the persona and their consulting impact.'}]
)
Prediction(
    reasoning='To identify the person known as "Aladin" in the context of organizational consulting, I need to conduct research to gather relevant information

In [18]:
class SelfGuidance(dspy.Module):
	def __init__(self):
		super().__init__()
		self.plan_and_guidance = dspy.Predict(Plan)
		self.tool = dspy.ChainOfThought(Tool)
		self.tool_ainvoke = tool_ainvoke
		self.cot = dspy.Predict(Result)
		self.final_answer = dspy.Predict(FinalAnswer)

	def forward(self, question: str):
		past_steps = []
		plan_result = self.plan_and_guidance(question=question)
		for step_and_guidance in plan_result.plan_and_guidance:
			tool_selection = self.tool(past_steps=past_steps, current_step_and_guidance=step_and_guidance)
			tool_result = asyncio.run(self.tool_ainvoke(tool_selection.tool))
			cot_result = self.cot(step=step_and_guidance["step"], guidance=step_and_guidance["guidance"],
			                      tool_result=tool_result)
			past_steps.append(
				{"step": step_and_guidance["step"], "evidence": cot_result.evidence, "result": cot_result.answer})
		final_answer = self.final_answer(question=question, steps=past_steps)
		return final_answer


In [25]:
import nest_asyncio

nest_asyncio.apply()
sg = SelfGuidance()
result = sg(question)

In [26]:
print(result)

Prediction(
    fusion="The investigation began with identifying Aladin as Aladin Jarrah, who holds roles as a communicator, strategic planner, and management consultant within the organizational consulting landscape. However, efforts to delve deeper into his specific consulting work yielded limited information, with no detailed accounts of projects, methodologies, or direct expertise in enhancing organizational performance found in the initial search results. The examination of his contributions indicated that while he may have influenced management practices and people's lives through idea shaping and legitimization, concrete evidence of his impact on organizational consulting remained elusive. The search also revealed no corroborative information that would validate his specific contributions or expertise as a management consultant, which implies a gap in publicly available knowledge regarding his direct influence and work in the field.",
    final_answer='Aladin, known as Aladin Ja